In [1]:
import pandas as pd

In [ ]:
df = pd.concat([pd.read_csv("maestro_vk.csv", index_col=0),
                pd.read_csv("maestro_blog.csv", index_col=0),
                pd.read_csv("maestro_books.csv", index_col=0)])

In [89]:
from razdel import sentenize


def sentify(df):
    df["sentence"] = df.text.apply(lambda y: [x.text for x in sentenize(y)])
    df = df.explode("sentence").reset_index(drop=True).drop("text", axis=1)
    return df

In [120]:
for df_name in [f"maestro_{i}.csv" for i in ["vk", "blog", "books"]]:
    df = pd.read_csv(df_name, index_col=0)
    sentify(df).to_csv(f"sent_{df_name}")

In [2]:
df = pd.concat([pd.read_csv("sent_maestro_vk.csv", index_col=0),
                pd.read_csv("sent_maestro_blog.csv", index_col=0),
                pd.read_csv("sent_maestro_books.csv", index_col=0)])

In [3]:
df = pd.read_csv("sent_combined.csv", index_col=0)
df.head()

,url,title,date,sentence
0,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,"Начну, простите, с конца."
1,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,"Да, митинги 10-го и 24-го декабря – это замеча..."
2,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,У России был один приближенный к серьезному ша...
3,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,"Немного снижая пафос, можно было бы вспомнить ..."
4,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,"Собственно на этом я бы и завершил статью, одн..."


In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'[^\d\W]+')

In [ ]:
class :
    def __init__(self, v=None):
        self.v = v

Быстрее всего искать что-либо в хешированных структурах, так что в идеале жолжен быть сет, либо хотя бы словрь. Словарь хорош тем, что в него можно сразу затолкать лемму и разбор слова

In [168]:
def preprocess(text):
    global i
    global df2

    lowered = tokenizer.tokenize(text.lower())
    forms = predictor.predict(lowered)
    clear_output()
    tokens = {}
    lemmas = {}
    poss = {}
    for form in forms:
        word = form.word
        lemma = form.normal_form
        pos = form.pos

        if word in tokens:
            tokens[word] += [i]
        else:
            tokens.update({word: [i]})

        if lemma in lemmas:
            lemmas[lemma] += [i]
        else:
            lemmas.update({lemma: [i]})
            
        if pos in poss:
            poss[pos] += [i]
        else:
            poss.update({pos: [i]})

        df2 = pd.concat([df2, pd.DataFrame({"token": word,
                                            "lemma": lemma,
                                            "pos": pos,
                                            "tag": form.tag
                                           },
                                            index=[i])])

        if i % 100 == 0:
            df2.to_csv("morph_infi.csv")
        i += 1

    return {"tokens": tokens, "lemmas": lemmas, "poss": poss}

In [ ]:
# в основном датафрейме храню только список токенов
# по нему будет искаться быстро, если нашёлся, беру номер предложения
# залетаю в токены, беру значение токена, это будет индекс в бд морфологии
# достаю из морфологии всю инфу

In [6]:
from IPython.display import clear_output
from tqdm import tqdm
tqdm.pandas()

In [320]:
i = 0
df2 = pd.DataFrame({"token":[], "lemma":[] , "pos":[], "tag":[]})

#t = pd.DataFrame(df.sentence.iloc[:10].progress_apply(preprocess).values)
t = pd.DataFrame(df.sentence.iloc[:100].progress_apply(preprocess).values)
t["lemmas"] = t[0].apply(lambda x: x["lemmas"])
t["tokens"] = t[0].apply(lambda x: x["tokens"])
t["poss"] = t[0].apply(lambda x: x["poss"])
t = t.drop(0, axis=1)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.80it/s]


In [171]:
t.iloc[0].poss

{'VERB': [0, 1], 'ADP': [2], 'NOUN': [3]}

In [322]:
t = pd.concat([df.iloc[:100][["title", "date", "sentence"]], t], axis=1)

In [143]:
word = "знать+VERB"

In [410]:
request = 'NOUN VERB'

In [339]:
"".isalnum()

True

In [336]:
import string

def isEnglish(s):
    return s.isascii()

In [483]:
def search(request, df):
    sub_req = request.split()

    el_chars = {}
    for element in sub_req:
        with_pos = False
        if element[0] == '"':
            element = element[1:-1]
            mode = "lemmas"
        elif element[0].isascii():
            mode = "poss"
        else:
            mode = "tokens"
        if "+" in element:
            with_pos = True
        print(element, mode)
        el_chars.update({element: word_search(element=element,
                                              df=df, mode=mode,
                                              with_pos=with_pos)})

    common_sents = list(reduce(lambda x, y: x & y.keys(), el_chars.values()))
    for sent in common_sents:
        # беру разницу элементов массива, она должна быть равна длине массива -1
        #word_indices = [el_chars[key][sent][0] for key in el_chars]
        listik = []
        for key in el_chars:
            listik.append(el_chars[key][sent])
        grid = np.array(np.meshgrid(*listik)).T.reshape(-1, len(el_chars))
        diffs = np.apply_along_axis(custom_diff, arr=grid, axis=1)
        word_indices = diffs[np.where(diffs == len(el_chars)-1)]
        if len(word_indices) > 0:
            print_out(sent, word_indices, df)

In [308]:
def print_out(sent_indice, word_indice, df):
    row = df.iloc[sent_indice]
    print(row.sentence)
    print(row.title)
    print(row.date)
    print("")

In [396]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [16]:
from functools import reduce

In [20]:
import numpy as np

In [478]:
grid = np.array(np.meshgrid([3], [0, 2])).T.reshape(-1, 2)
custom_diff = lambda z: reduce(lambda x, y: abs(x - y), z)
diffs = np.apply_along_axis(custom_diff, arr=grid, axis=1)
diffs[np.where(diffs == 1)]

array([1])

In [476]:
diffs

array([3, 2])

In [484]:
search(request, df=t)

NOUN poss
VERB poss
У России был один приближенный к серьезному шанс на вменяемое развитие в начале девяностых, но действовавшие тогда поколения допустили две фатальные ошибки: во-первых, не была проведена люстрация (как сделали чехи и поляки) и, во-вторых, вместо выноса архива КГБ (как это сделали с архивом Штази немцы) народ удовольствовался тем, что ему дали поиграть куском металла в виде памятника железному Феликсу, который из-за этих просчетов оказался фениксом.
Кутузовский план Кремля (27. 12. 2011). Запись в ВК
2021-02-24

Немного снижая пафос, можно было бы вспомнить и про второй шанс, когда летом 2010 года Кремль исчез в дыму, и все чиновники с охранниками спасали свое здоровье вне кабинетов, но не до шуток нынче…
Кутузовский план Кремля (27. 12. 2011). Запись в ВК
2021-02-24

Собственно на этом я бы и завершил статью, однако есть еще много интересных наблюдений и открытий.
Кутузовский план Кремля (27. 12. 2011). Запись в ВК
2021-02-24

Как автор монографии «Правда о войне 181

In [407]:
a = np.array([1, 2, 3, 4])
np.abs(np.subtract.outer(a, a))

array([[0, 1, 2, 3],
       [1, 0, 1, 2],
       [2, 1, 0, 1],
       [3, 2, 1, 0]])

In [ ]:
pd.merge(pd.DataFrame(word_search("не", t, mode="tokens")),
         pd.DataFrame(word_search("знать", t, mode="lemmas")))

In [328]:
def word_search(element, df=df, df2=df2, mode="tokens", with_pos=False):

    if with_pos:
        word, pos = element.split("+")
        word_indices = word_search(word, df, df2, mode)
        pos_indices = word_search(pos, df, df2, mode="poss")
        indices = word_indices.keys() & pos_indices.keys()
        morph_indices = []
        for sent in indices:
            morph_indices.append(list(set(word_indices[sent]) & set(pos_indices[sent])))

    else:
        v = np.vectorize(lambda x: element in x)
        indices = np.where(v(df[mode]) == True)[0]

        #for i in indices:
            #print(i)
        row = df.iloc[indices]
        #print(row.lemmas)
        #print(row.sentence)
        #print(row.title)
        #print(row.date)
        morph_indices = row[mode].apply(lambda x: x[element])
        if with_pos:
        #print(row[mode].apply(lambda x: x[word]))  # индексы находятся списками
            morph = df2.iloc[morph_indices.sum()]
            u = np.vectorize(lambda x: x == pos)

            #print(f"""\tnormal form - {morph.lemma.unique()}
            #word - {morph.token.unique()}
            #pos - {morph.pos.unique()}
            #tag - {morph.tag.unique()}""")
            #print("")
        
    return dict(zip(indices, morph_indices))

In [233]:
df3 = pd.concat([
           pd.DataFrame({"url": "test0",
                         "title": "test1",
                         "date": "test2",
                         "sentence": "сегодня тут ужинает высшая знать"}, index=[0]),
    df])

In [330]:
word_search("знать+VERB", t, mode="tokens", with_pos=True)

{87: [2026]}

In [345]:
word_search("VERB", t, mode="poss", with_pos=False)

{0: [0, 1],
 1: [25, 34, 35, 41],
 2: [44, 60, 67, 71, 84, 90, 94, 95, 108],
 3: [111, 114, 116, 125, 133],
 4: [149, 152],
 6: [171],
 7: [175, 178, 180, 188, 200, 206, 210, 217, 221, 231, 232, 236, 238],
 8: [251],
 9: [270, 281, 283],
 10: [289, 292, 295],
 11: [301, 306, 324, 326, 336, 347],
 12: [361, 369, 372, 377, 379, 391, 399, 402, 405],
 13: [407, 418, 431, 433, 443, 451, 457, 467, 468, 473, 480, 482],
 14: [494, 495, 500, 501, 513, 519, 521],
 16: [529, 530, 534],
 17: [541, 542],
 18: [559, 560, 572, 574, 583, 596, 598],
 19: [615],
 21: [633, 638],
 22: [645, 647, 651, 656],
 23: [659, 662],
 24: [672, 680, 683, 685, 701, 704],
 25: [709, 715, 729, 734, 742, 745, 754],
 26: [756, 757, 760, 775],
 28: [783],
 29: [792],
 30: [798, 807, 812, 815, 820, 825, 828],
 31: [831],
 32: [851],
 34: [860, 871],
 35: [876, 878, 884],
 36: [897, 913],
 37: [934],
 38: [945, 946, 952, 963, 964, 979],
 39: [980, 981, 984, 988, 992, 995],
 40: [1004],
 41: [1009, 1025],
 43: [1038, 1039, 

In [329]:
word_search("знать+NOUN", t, mode="tokens", with_pos=True)

{87: []}

In [331]:
word_search("знать", t, mode="tokens", with_pos=False)

{87: [2026]}

In [651]:
print(df.iloc[398]["sentence"])

Конечно, не обошлось и без дипломированных лицедеев – актрисы Натальи Селезневой и актера Валерия Баринова.


In [867]:
a, b, c, d = word_search(word, t, mode="lemmas")

9      [281]
87    [2026]
Name: lemmas, dtype: object


In [627]:
%%time
v = np.vectorize(lambda x: word in x)

indices = np.where(v(t["tokens"]) == True)

t.iloc[indices]["tokens"].apply(lambda x: df2.iloc[x[word]].values[0])
#chars = s.iloc[indices]["lemmas"].apply(lambda x: x[word])

CPU times: total: 0 ns
Wall time: 998 µs


0    [начну, начать, VERB, Mood=Ind|Number=Sing|Per...
Name: tokens, dtype: object

In [647]:
j = [0, 1]

for i in j:
    print(df.iloc[i]["sentence"])
    morph = df2.iloc[i]
    print(f"""\tnormal form - {morph.lemma}
        pos - {morph.pos}
        tag - {morph.tag}""")

Начну, простите, с конца.
	normal form - начать
        pos - VERB
        tag - Mood=Ind|Number=Sing|Person=1|Tense=Notpast|VerbForm=Fin|Voice=Act
Да, митинги 10-го и 24-го декабря – это замечательная веха отечественной истории, да, прежде всего, это дыхание весны и романтики (я вообще вижу в них, прежде всего, художественный смысл), но (не хочу оказаться правым) ОНИ вам ничего не отдадут.
	normal form - простить
        pos - VERB
        tag - Mood=Imp|Number=Plur|Person=2|VerbForm=Fin|Voice=Act


In [622]:
t.iloc[indices]["tokens"].apply(lambda x: df2["token"].iloc[x[word]])

,84
0,начну


In [464]:
s = pd.concat([df.iloc[:1000].sentence, s], axis=1)

In [465]:
v = np.vectorize(lambda x: (x["pos"], x["tag"]))
v(chars.values)

(array(['VERB', 'VERB', 'VERB', 'VERB', 'VERB', 'VERB', 'VERB', 'VERB',
        'VERB', 'VERB', 'VERB'], dtype='<U4'),
 array(['Mood=Ind|Number=Sing|Person=1|Tense=Notpast|VerbForm=Fin|Voice=Act',
        'Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act',
        'Mood=Ind|Number=Plur|Person=3|Tense=Notpast|VerbForm=Fin|Voice=Act',
        'Mood=Ind|Number=Sing|Person=1|Tense=Notpast|VerbForm=Fin|Voice=Act',
        'Mood=Ind|Number=Plur|Person=1|Tense=Notpast|VerbForm=Fin|Voice=Act',
        'Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act',
        'Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act',
        'Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act',
        'VerbForm=Inf',
        'Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act',
        'Mood=Ind|Number=Sing|Person=3|Tense=Notpast|VerbForm=Fin|Voice=Act'],
       dtype='<U66'))

In [ ]:
3 ms - занимает поиск

In [ ]:
# 4 колонки, сначала ищу по датафрейму, это супербыстро с векторайзом, потом в каждой строке ищу индекс один раз, вывожу всю инфу из 4 колонок

In [ ]:
#apply - 188 ms vs vectorize - 15 ms
# подход со словарём уоскрил в 3 раза

In [376]:
indices

(array([0], dtype=int64),)

In [406]:
word = "да"

In [432]:
print(s[["tokens"]].iloc[indices]["tokens"].apply(lambda x: x[word]))
print("!")

1     {'pos': 'CONJ', 'tag': '_'}
12    {'pos': 'CONJ', 'tag': '_'}
22    {'pos': 'CONJ', 'tag': '_'}
30    {'pos': 'CONJ', 'tag': '_'}
47    {'pos': 'CONJ', 'tag': '_'}
Name: tokens, dtype: object
!


In [386]:
# думаю, что так лучше всего, тогда можно по индексу сразу найти предложение, его вывести
# в соотв колонке сразу обратиться по ключу, достать морфоприколы + лемму
# но нужно сделать что-то подобное с другим словарём тоже

# или сделать другую бд, но искать-же всё равно надо по хэшу, а где мне его ещё достать
# обязательно должно что-то на что-то ссылаться
# наверное надо всё-таки хранить лемму в токене тоже и наоборот
# токены надо хранить в сете, иначе я их буду долго искать

# судя по всему список позов тоже нужно хранить, но там нужен только сет,
# чтобы достать форму+поз я посмотрю по словарю


А если всё-таки всё разбить? Искать слово. Допустим, нашёл, вернул индекс, написал предложение + всё, что просят. Вроде так максимально просто. Если поиск в пандасе такой же быстрый как в словаре

In [8]:
from rnnmorph.predictor import RNNMorphPredictor

predictor = RNNMorphPredictor(language="ru")

In [204]:
forms = predictor.predict(["мама", "мыла", "раму"])
print(forms[0].pos)
print(forms[0].tag)
print(forms[0].normal_form)
forms[0].word

1/1 [==============================] - 0s 20ms/step
NOUN
Case=Nom|Gender=Fem|Number=Sing
мама


'мама'